# [Day 18] 첫 번째 손가락 제어: 웹캠 연동

어제 트랙바를 통해 로봇의 한계를 잘 파악하셨나요? 오늘은 드디어 **카메라**를 연결하여 내 손의 움직임에 따라 로봇이 움직이도록 만들어보겠습니다.

1. 카메라로 손 인식 (Day 9)
2. 각도 계산 (Day 13)
3. 모터 각도로 변환 (Day 14)
4. 명령어로 포장 (Day 16)
5. 시리얼 전송 (Day 15)

오늘은 욕심내지 않고 **검지 손가락 하나**만 내 손을 따라 움직이게 해보겠습니다.

In [1]:
# 포트 선택 도구 가져오기
import serial.tools.list_ports
import ipywidgets as widgets
from IPython.display import display

# 1. 컴퓨터에 연결된 시리얼 포트 찾기
ports = serial.tools.list_ports.comports()
available_ports = [port.device for port in ports]

if not available_ports:
    print("감지된 포트가 없습니다. 임시로 COM1을 목록에 추가합니다.")
    available_ports = ['COM1']

# 2. 드롭다운 메뉴 만들기
port_dropdown = widgets.Dropdown(
    options=available_ports,
    value=available_ports[0],
    description='포트 선택:',
    disabled=False,
    )

print("아두이노가 연결된 포트를 선택해주세요:")
display(port_dropdown)

아두이노가 연결된 포트를 선택해주세요:


Dropdown(description='포트 선택:', options=('COM8', 'COM9'), value='COM8')

In [ ]:
# 필요한 라이브러리와 함수들을 모두 가져옵니다.
import cv2 as cv
import mediapipe as mp
import math
import serial
import time

# ----- 설정 -----  
# 위에서 선택한 포트 값을 가져옵니다.
PORT = port_dropdown.value
ser = None

try:
    ser = serial.Serial(PORT, 115200)
    print(f"로봇 연결 성공! ({PORT})")
except:
    print(f"로봇 연결 실패 ({PORT}) - 시뮬레이션 모드로 진행합니다")

# 수학 함수들 (생략하지 않고 다시 정의해줍니다)
def vector_length(v):
    return math.sqrt(v[0]**2 + v[1]**2 + v[2]**2)
def dot_product(v1, v2):
    return (v1[0]*v2[0]) + (v1[1]*v2[1]) + (v1[2]*v2[2])
def angle_between(v1, v2):
    den = vector_length(v1) * vector_length(v2)
    if den == 0: return 0
    return math.degrees(math.acos(max(-1.0, min(1.0, dot_product(v1, v2) / den))))
def map_value(x, in_min, in_max, out_min, out_max):
    return (max(min(x, in_max), in_min) - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

# ----- 메인 루프 -----  
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils
cap = cv.VideoCapture(0)

last_send_time = 0 # 통신 속도 조절용

while True:
    ret, frame = cap.read()
    if not ret: break
    frame = cv.flip(frame, 1)
    image_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    
    if results.multi_hand_world_landmarks:
        for hl in results.multi_hand_world_landmarks:
            # 1. 검지 각도 계산 (0-5-8)
            p0 = hl.landmark[0]
            p5 = hl.landmark[5]
            p8 = hl.landmark[8]
            
            # 벡터 생성
            v1 = [p0.x-p5.x, p0.y-p5.y, p0.z-p5.z]
            v2 = [p8.x-p5.x, p8.y-p5.y, p8.z-p5.z]
            
            angle = angle_between(v1, v2)
            
            # 2. 모터 각도로 변환 (내 손: 30~160 -> 로봇: 180~0)
            # 검지만 움직일 것이므로 나머지는 0으로 둡니다.
            motor_angle = int(map_value(angle, 30, 160, 180, 0))
            
            # 3. 데이터 전송 (너무 자주 보내면 렉 걸림)
            if time.time() - last_send_time > 0.1: # 0.1초마다 전송
                # 패킷 생성: FR0[000]1[검지]2[000]3[000]4[000]
                packet = f"FR00001{motor_angle:03d}200030004000\n"
                
                if ser:
                    ser.write(packet.encode())
                
                print(f"Angle: {int(angle)} -> Send: {packet.strip()}")
                last_send_time = time.time()

            cv.putText(frame, f"Index: {int(angle)}", (10, 50), 
                       cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # 화면 그리기용 랜드마크
    if results.multi_hand_landmarks:
        for hl in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hl, mp_hands.HAND_CONNECTIONS)
            
    cv.imshow('Robot Control', frame)
    if cv.waitKey(1) == ord('q'):
        break

cap.release()
if ser: ser.close()
cv.destroyAllWindows()

## 축하합니다!

방금 여러분은 가상 세계(파이썬 코드)와 현실 세계(로봇)를 연결했습니다.
검지 손가락이 움직이는 것을 보셨나요?

혹시 움직이지 않는다면 다음을 체크해보세요.
1. **포트 번호**가 맞나요?
2. **배터리 전원**이 켜져 있나요? (USB 전원만으로는 모터가 안 움직일 수 있습니다)
3. **각도 범위(30~160)**가 내 손과 맞나요?

내일은 나머지 네 손가락도 모두 깨워보겠습니다.